In [ ]:
import math
import os
import textwrap

import keras
import numpy as np
import pandas as pd
# import pandas_profiling
import statsmodels.api as sm
import tensorflow as tf
import tensorflow_addons as tfa
import xgboost as xgb
from keras.constraints import maxnorm
from keras.layers import Dense
from keras.layers import Dropout
from keras.models import Sequential
from keras.optimizers import Adam
from keras.wrappers.scikit_learn import KerasRegressor
from keras.wrappers.scikit_learn import KerasRegressor
from matplotlib import pyplot as plt
from numpy import absolute, mean, std
from sklearn import linear_model
from sklearn.ensemble import (
    AdaBoostRegressor,
    BaggingRegressor,
    ExtraTreesRegressor,
    GradientBoostingRegressor,
    RandomForestRegressor,
)
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.model_selection import (
    GridSearchCV,
    RepeatedKFold,
    cross_validate,
    train_test_split,
)
from sklearn.model_selection import ParameterGrid
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR, LinearSVR, NuSVR
from sklearn.tree import DecisionTreeRegressor
# from COGStats import batchQQCheck
from statsmodels.regression.quantile_regression import QuantReg

# from COGStats import *

In [ ]:
%matplotlib inline

In [ ]:
plot_path = "ML_plots_livewebmdata//"

In [ ]:
isExist = os.path.exists(plot_path)

if not isExist:

    # Create a new directory because it does not exist
    os.makedirs(plot_path)


isExist = os.path.exists(plot_path + "_scaled//")

if not isExist:

    # Create a new directory because it does not exist
    os.makedirs(plot_path + "_scaled//")

In [ ]:
f = open("livewebmdata.txt", "r", encoding="utf-8")

#Logs of prior conversion in the form:
    #File_name: raw_video_file.avi
    #initial_bitrate: 1897140.707401512
    #orig_bitrate: 170410473.0
    #avcCodec: 0.0
    #mpjgCodec: 1.0
    #height: 2130.0
    #width: 3840.0
    #duration: 6.847458
    #frames: 404.0
    #origSize: 145859820.0


lines = f.readlines()

print(len(lines))
f.close()

duration = []
frames = []
original_filesize = []
output_filesize = []
initial_bitrate = []
orig_bitrate = []
height = []
width = []
avc_Codec = []
mpjg_Codec = []


for line in lines:
    if (
        "size:" in line
        or "orig_bitrate" in line
        or "initial_bitrate" in line
        or "frames:" in line
        or "origSize:" in line
        or "orig_bitrate:" in line
        or "duration:" in line
        or "avcCodec:" in line
        or "mpjgCodec:" in line
        or "height:" in line
        or "width:" in line
        or "origSize:" in line
        or "output file size:" in line
    ):

        # print(line.split()[-1])
        if "initial_bitrate:" in line:
            initial_bitrate.append(float(line.split()[-1]))

        if "orig_bitrate:" in line:
            orig_bitrate.append(float(line.split()[-1]))

        if "avcCodec:" in line:
            avc_Codec.append(float(line.split()[-1]))

        if "mpjgCodec:" in line:
            mpjg_Codec.append(float(line.split()[-1]))

        if "height:" in line:
            height.append(float(line.split()[-1]))

        if "width:" in line:
            width.append(float(line.split()[-1]))

        if "duration:" in line:
            duration.append(float(line.split()[-1]))

        if "frames:" in line:
            frames.append(float(line.split()[-1]))

        if "origSize:" in line:
            original_filesize.append(float(line.split()[-1]))

        if "output file size:" in line:
            output_filesize.append(float(line.split()[-1]))


df = pd.DataFrame(
    columns=[
        "initial_bitrate",
        "orig_bitrate",
        "avc_Codec",
        "mpjg_Codec",
        "height",
        "width",
        "duration",
        "frames",
        "original_filesize",
        "output_filesize",
    ]
)

df["duration"] = duration
df["frames"] = frames
df["original_filesize"] = original_filesize
df["orig_bitrate"] = orig_bitrate
df["initial_bitrate"] = initial_bitrate
df["height"] = height
df["width"] = width
df["avc_Codec"] = avc_Codec
df["mpjg_Codec"] = mpjg_Codec
df["output_filesize"] = output_filesize

df.describe()


X = df[
    [
        "initial_bitrate",
        "orig_bitrate",
        #"avc_Codec",
        #"mpjg_Codec",
        #"height",
        #"width",
        "duration",
        #"frames",
        "original_filesize",
    ]
].copy()
y = df["output_filesize"].copy()

## fit a OLS model with intercept on TV and Radio
X = sm.add_constant(X, has_constant="add")
est = sm.OLS(y, X).fit()

In [ ]:
# df = df.sample(2500)

In [ ]:
df.columns

In [ ]:
# proile = pandas_profiling.ProfileReport(df)

In [ ]:
# profile.to_file(output_file='Pandas Profiling Report — all_samples4 .html')

In [ ]:
df.describe()

In [ ]:
df.describe()

In [ ]:
df.columns

In [ ]:
df.head()

In [ ]:
scale = False
col_names = df.columns
if scale == True:
    scaler = StandardScaler()
    # transform data
    df = scaler.fit_transform(df)
    df = pd.DataFrame(df, columns=col_names)
    plot_path = plot_path + "_scaled//"

In [ ]:
df.head()

In [ ]:
for col in df.columns:
    try:
        df[col].plot(kind="density")

    except Exception as e:
        print(col)
        print(e)

In [ ]:
X = df[
    [
        "initial_bitrate",
        "orig_bitrate",
        #        "height",
        #        "width",
        "duration",
        #        "frames",
        "original_filesize",
    ]
]
y = df["output_filesize"]

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# Common args

In [ ]:
cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)

In [ ]:
metric_list = ["neg_mean_absolute_error", "r2"]

In [ ]:
scores_tuple_list = []

In [ ]:
def grid_search_crossvalidate(
    algorithm_implementation, parameter_space, method_name, X, y
):
    global scores_tuple_list

    print(method_name)

    current_algorithm = algorithm_implementation

    parameters = parameter_space

    grid = GridSearchCV(
        current_algorithm, parameters, verbose=3, cv=[(slice(None), slice(None))]
    )  # Negate CV in param search for speed

    pg = ParameterGrid(parameter_space)
    pg = len(pg)
    print(pg)
    if pg > 100000:
        print("grid too large", str(pg))
        raise Exception("grid too large", str(pg))

    grid.fit(X, y)
    print(grid)

    if type(grid.estimator) is not keras.wrappers.scikit_learn.KerasRegressor:

        current_algorithm = grid.best_estimator_.fit(X, y)
    else:
        current_algorithm = KerasRegressor(
            build_fn=create_model,
            verbose=0,
            layers=grid.best_params_["layers"],
            width=grid.best_params_["width"],
            learning_rate=grid.best_params_["learning_rate"],
        )

    # current_algorithm.score(X, y)

    scores = cross_validate(
        current_algorithm,
        X,
        y,
        scoring=metric_list,
        cv=cv,  # Full CV on final best model
        n_jobs=1,
        verbose=0,
    )
    current_algorithm_scores = scores
    scores_tuple_list.append((method_name, current_algorithm_scores, grid))

    print(
        "Mean MAE: %.3f (%.3f)"
        % (
            absolute(mean(scores["test_neg_mean_absolute_error"])),
            std(scores["test_neg_mean_absolute_error"]),
        )
    )
    print(
        "Mean R2: %.3f (%.3f)"
        % (absolute(mean(scores["test_r2"])), std(scores["test_r2"]))
    )
    print(
        "Mean fit time: %.3f (%.3f)"
        % (absolute(mean(scores["fit_time"])), std(scores["fit_time"]))
    )
    print(
        "Mean score time: %.3f (%.3f)"
        % (absolute(mean(scores["score_time"])), std(scores["score_time"]))
    )
    print(
        "---------------------------------------------------------------------------------------------------"
    )


    y_predict = grid.best_estimator_.predict(x_test)

    plt.figure(figsize=(10, 10))
    plt.tight_layout()
    fig, ax = plt.subplots()
    ax.scatter(y_test, y_predict, c="crimson")
    # plt.yscale('log')
    # plt.xscale('log')

    p1 = max(max(y_predict), max(y_test))
    p2 = min(min(y_predict), min(y_test))
    plt.plot([p1, p2], [p1, p2], "b-")
    ax.set_xlabel("True Values", fontsize=15)
    ax.set_ylabel("Predictions", fontsize=15)
    plt.axis("equal")
    # ax.set_title(str(current_algorithm)+ 'R2: ' + str(round(r2_score(y_test, y_predict),3)) +'\n' + str(current_algorithm.get_params()))
    # ax.set_title(str(current_algorithm)+ ' R2: ' + str(round(absolute(mean(scores["test_r2"])),3)) +'\n' + str(current_algorithm.get_params()))

    try:
        print(
            textwrap.fill(
                str(current_algorithm)
                + str(pg)
                + "_splits."
                + " R2: "
                + str(round(absolute(mean(scores["test_r2"])), 3))
                + str(current_algorithm.get_params()),
                60,
            )
        )
    except Exception as e:
        print(e, "failed to plot")

    ax.set_title(
        textwrap.fill(
            str(method_name)
            + str(pg)
            + "_splits."
            + " R2: "
            + str(round(absolute(mean(scores["test_r2"])), 3))
            + str(current_algorithm.get_params()),
            60,
        )
    )
    # ax.annotate("r-squared = {:.3f}".format(r2_score(y_test, y_predict)), (0, 1))
    # plt.annotate("r-squared = {:.3f}".format(r2_score(y_test, y_predict)), (0, 1))
    # plt.show()
    plt.savefig(
        plot_path + str(method_name).split("(")[0] + "_reg.png",
        edgecolor="black",
        dpi=150,
        transparent=True,
        bbox_inches="tight",
    )

    return (method_name, current_algorithm_scores)

class_name = kerasRegressor_regression

grid_search_crossvalidate(class_name.algorithm_implementation,
                         class_name.parameter_space,
                         class_name.method_name,
                         class_name.X_data,
                         class_name.y_data);

In [ ]:
# class_name = multiple_regression

# grid_search_crossvalidate(class_name.algorithm_implementation,
#                          class_name.parameter_space,
#                          class_name.method_name,
#                          class_name.X_data,
#                          class_name.y_data);

In [ ]:
log_small = np.logspace(-1, -5, 3)
print(log_small)
bool_param = [True, False]
print(bool_param)
log_large = np.logspace(0, 2, 3).astype(int)
print(log_large)
log_large_long = np.floor(np.logspace(0, 2.5, 5)).astype(int)
print(log_large_long)
log_med_long = np.floor(np.logspace(0, 1.5, 5)).astype(int)
print(log_med_long)
log_med = np.floor(np.logspace(0, 1.5, 3)).astype(int)
print(log_med)
nstep = 3
log_zero_one = np.logspace(0.0, 1.0, nstep) / 10
print(log_zero_one)
lin_zero_one = np.linspace(0.0, 1.0, nstep) / 10
print(lin_zero_one)

In [ ]:
class multiple_regression:
    "multiple_regression"
    algorithm_implementation = LinearRegression()
    parameter_space = {}  # {'fit_intercept':[False]}
    method_name = "multiple_regression"
    X_data = X
    y_data = y
    parameter_space = {}

In [ ]:
class ridge_regression:
    "Ridge"
    algorithm_implementation = Ridge()

    method_name = "Ridge"
    X_data = X
    y_data = y
    parameter_space = {
        "alpha": log_small,
        "copy_X": [True],
        "max_iter": log_large,
        "tol": log_small,
        "solver": [
            "auto",
            "svd",
            "cholesky",
            "lsqr",
            "sparse_cg",
            "sag",
            "saga",
            "lbfgs",
        ],
        "positive": bool_param,
        "random_state": bool_param,
    }

In [ ]:
class lasso_regression:
    "Lasso"
    algorithm_implementation = linear_model.Lasso()
    method_name = "Lasso"
    X_data = X
    y_data = y
    parameter_space = {
        "alpha": log_small,
        "fit_intercept": [True],
        "copy_X": [True],
        "max_iter": log_large,
        "tol": log_small,
        "positive": bool_param,
        "selection": ["cyclic", "random"],
    }

In [ ]:
class linearSVR_regression:
    "LinearSVR"
    algorithm_implementation = LinearSVR()
    method_name = "LinearSVR"
    X_data = X
    y_data = y
    parameter_space = {
        "C": log_small,
        "dual": [True],
        "epsilon": log_small,
        "fit_intercept": [True],
        #'intercept_scaling': 1.0,
        "loss": ["epsilon_insensitive", "squared_epsilon_insensitive"],
        "max_iter": [1000],
        "random_state": [None],
        "tol": log_small,
        "verbose": [0],
    }

In [ ]:
class baggingregressor_regression:
    "BaggingRegressor"
    algorithm_implementation = BaggingRegressor()
    method_name = "BaggingRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "n_estimators": log_large_long,
        "max_samples": log_large_long,
        "max_features": log_large_long,
        "bootstrap": bool_param,
        "bootstrap_features": bool_param,
        "oob_score": bool_param,
    }

In [ ]:
class kneighborsregressor_regression:
    "KNeighborsRegressor"
    algorithm_implementation = KNeighborsRegressor()
    method_name = "KNeighborsRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "algorithm": ["auto", "ball_tree", "kd_tree", "brute"],
        "leaf_size": log_large_long,
        "metric": ["minkowski"],
        "metric_params": [None],
        "n_jobs": [None],
        "n_neighbors": log_med_long,
        "p": [2],
        "weights": ["uniform", "distance"],
    }

In [ ]:
class randomforestregressor_regression:
    "RandomForestRegressor"
    algorithm_implementation = RandomForestRegressor()
    method_name = "RandomForestRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "bootstrap": bool_param,
        "ccp_alpha": log_small,
        "criterion": ["squared_error", "absolute_error", "poisson"],
        "max_depth": log_large_long,
        "max_features": ["auto", "sqrt", "log2", None],
        #'max_leaf_nodes': log_large_long,
        "max_samples": log_large_long,
        #'min_impurity_decrease': log_small,
        #'min_samples_leaf': log_med,
        #'min_samples_split': log_med,
        #'min_weight_fraction_leaf': log_small,
        "n_estimators": log_large_long,
        "n_jobs": [None],
        "oob_score": bool_param,
        #'random_state': bool_param,
        "verbose": [0],
        #'warm_start': bool_param}
    }

In [ ]:
class gradientboostingregressor_regression:
    "GradientBoostingRegressor"
    algorithm_implementation = GradientBoostingRegressor()
    method_name = "GradientBoostingRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "alpha": log_small,
        "ccp_alpha": log_small,
        "criterion": ["friedman_mse"],
        "init": [None],
        "learning_rate": log_small,
        "loss": ["squared_error", "huber"],
        #'max_depth': log_med,
        "max_features": ["auto", "sqrt", "log2", None],
        #'max_leaf_nodes': [None],
        #'min_impurity_decrease': log_small,
        #'min_samples_leaf':log_med,
        #'min_samples_split': log_med,
        #'min_weight_fraction_leaf': log_small,
        "n_estimators": log_large_long,
        "n_iter_no_change": [None],
        "random_state": [None],
        #'subsample': log_small,
        "tol": log_small,
        "validation_fraction": [0.1],
        "verbose": [0],
        #'warm_start': bool_param
    }

In [ ]:
class decisiontreeregressor_regression:
    "DecisionTreeRegressor"
    algorithm_implementation = DecisionTreeRegressor()
    method_name = "DecisionTreeRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "ccp_alpha": log_small,
        "criterion": ["squared_error", "friedman_mse", "poisson"],
        "max_depth": log_large_long,
        "max_features": log_med,
        "max_leaf_nodes": log_med,
        #'min_impurity_decrease': log_small,
        #'min_samples_leaf': log_med,
        #'min_samples_split': log_med,
        #'min_weight_fraction_leaf': log_small,
        "random_state": [None],
        "splitter": ["best", "random"],
    }

In [ ]:
class SVR_regression:
    "SVR"
    algorithm_implementation = SVR()
    method_name = "SVR"
    X_data = X
    y_data = y
    parameter_space = {
        "C": lin_zero_one,
        "cache_size": [200],
        "coef0": log_small,
        "degree": log_med,
        "epsilon": log_small,
        "gamma": ["scale"],
        "kernel": ["rbf", "linear", "poly", "sigmoid"],#, "precomputed"],
        "max_iter": log_large_long,
        "shrinking": bool_param,
        "tol": log_small,
        "verbose": [0],
    }

In [ ]:
class NuSVR_regression:
    "NuSVR"
    algorithm_implementation = NuSVR()
    method_name = "NuSVR"
    X_data = X
    y_data = y
    parameter_space = {
        "C": lin_zero_one,
        "cache_size": [200],
        "coef0": log_small,
        "degree": log_med,
        "gamma": ["scale"],
        "kernel": ["rbf", "linear", "poly", "sigmoid"],# "precomputed"],
        "max_iter": [-1],
        "nu": log_zero_one,
        "shrinking": bool_param,
        "tol": log_small,
        "verbose": [0],
    }

In [ ]:
class MLPRegressor_regression:
    "MLPRegressor"
    algorithm_implementation = MLPRegressor()
    method_name = "MLPRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "activation": ["relu", "tanh"],
        "alpha": log_small,
        #'batch_size': ['auto']+list(log_large_long),
        "batch_size": ["auto"],
        #'beta_1': 0.9,
        #'beta_2': 0.999,
        "early_stopping": [False],
        #'epsilon': 1e-08,
        "hidden_layer_sizes": log_large_long,
        "learning_rate": ["constant", "adaptive"],
        "learning_rate_init": log_small,
        #'max_fun': 15000,
        "max_iter": [300],
        "momentum": log_small,
        "n_iter_no_change": [10],
        #'nesterovs_momentum': True,
        #'power_t': 0.5,
        "random_state": [None],
        "shuffle": bool_param,
        #'solver': ['adam', 'lbfgs', 'sgd'],
        "tol": log_small,
        "validation_fraction": [0.1],
        "verbose": [0],
        "warm_start": [False],
    }

In [ ]:
class SVR_regression:
    "SVR"
    algorithm_implementation = SVR()
    method_name = "SVR"
    X_data = X
    y_data = y
    parameter_space = {
        "C": lin_zero_one,
        "cache_size": [200],
        "coef0": log_small,
        "degree": log_med,
        "epsilon": lin_zero_one,
        "gamma": ["scale"],
        "kernel": ["rbf", "linear", "poly", "sigmoid"],#, "precomputed"],
        "max_iter": [-1],
        "shrinking": bool_param,
        "tol": log_small,
        "verbose": [0],
    }

In [ ]:
class AdaBoostRegressor_regression:
    "AdaBoostRegressor"
    algorithm_implementation = AdaBoostRegressor()
    method_name = "AdaBoostRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "base_estimator": [None],
        "learning_rate": log_small,
        "loss": ["linear", "square", "exponential"],
        "n_estimators": log_large_long,
        "random_state": [None],
    }

In [ ]:
class QuantileRegressor_regression:
    "QuantileRegressor"
    quantiles = [0.5]

    method_name = "QuantileRegressor"
    X_data = X
    y_data = y
    algorithm_implementation = QuantReg(X_data, y_data)
    parameter_space = {"quantile": [0.5], "alpha": log_small}

In [ ]:
class ExtraTreesRegressor_regression:
    "ExtraTreesRegressor"
    algorithm_implementation = ExtraTreesRegressor()
    method_name = "ExtraTreesRegressor"
    X_data = X
    y_data = y
    parameter_space = {
        "bootstrap": bool_param,
        "ccp_alpha": log_small,
        "criterion": ["squared_error"],
        "max_depth": [None] + list(log_large_long),
        "max_features": ["auto"],
        # 'max_leaf_nodes': log_med,
        #'max_samples': log_med,
        #'min_impurity_decrease': log_small,
        #'min_samples_leaf': log_med,
        #'min_samples_split': log_med,
        #'min_weight_fraction_leaf': log_small,
        "n_estimators": log_large_long,
        "n_jobs": [None],
        "oob_score": [False],
        "random_state": [None],
        "verbose": [0],
        "warm_start": [False],
    }

In [ ]:
class XGBRegressor_linear_regression:
    "XGBRegressor_linear_regression"
    algorithm_implementation = xgb.XGBRegressor()
    method_name = "XGBRegressor_regression"
    X_data = X
    y_data = y

    parameter_space = {
        "n_estimators": log_large_long,
        "learning_rate": log_small,
        "reg_lambda": log_small,
        "reg_alpha": log_small,
        "tree_method": ["gpu_hist"],
        "max_depth": log_large_long,
        "max_leaves": log_large_long,
        "grow_policy": [0, 1],
        "booster": ["gblinear"],
        "n_jobs": [4],
    }

In [ ]:
class XGBRegressor_tree_regression:
    "XGBRegressor_tree_regression"
    algorithm_implementation = xgb.XGBRegressor()
    method_name = "XGBRegressor_regression"
    X_data = X
    y_data = y

    parameter_space = {
        "n_estimators": log_large_long,
        "learning_rate": log_small,
        "reg_lambda": log_small,
        "reg_alpha": log_small,
        "tree_method": ["gpu_hist"],
        "max_depth": log_large_long,
        "max_leaves": log_large_long,
        "grow_policy": ['depthwise', 'lossguide'],
        "booster": ["gbtree"],
        "n_jobs": [4],
    }

In [ ]:
def create_model(
    layers=1,
    kernel_reg=tf.keras.regularizers.l1_l2(l1=0, l2=0),
    width=15,
    learning_rate=0.01,
    dropout_val=0.2,
    input_dim_val=len(X.columns),
):

    # create model
    model = Sequential()
    for i in range(0, layers):
        model.add(
            Dense(
                math.floor(width),
                input_dim=len(X.columns),
                kernel_initializer="normal",
                activation="relu",
                kernel_constraint=maxnorm(4),
                kernel_regularizer=kernel_reg,
            )
        )

    model.add(Dropout(dropout_val))
    model.add(Dense(1, kernel_initializer="normal", activation="relu"))
    # Compile model
    optimizer = Adam(learning_rate=learning_rate)

    metric = tfa.metrics.r_square.RSquare()

    model.compile(
        loss="mean_squared_error",
        optimizer=optimizer,
        metrics=[metric, "mean_squared_error", rmse],
    )

    # print(kernel_reg.get_config())
    return model

In [ ]:
from keras import backend as K


def coeff_determination(y_true, y_pred):
    SS_res = K.sum(K.square(y_true - y_pred))
    SS_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - SS_res / (SS_tot + K.epsilon())


def coeff_determinationn(y_true, y_pred):
    SS_res = np.sum(np.square(y_true - y_pred))
    SS_tot = np.sum(K.square(y_true - np.mean(y_true)))
    return 1 - SS_res / (SS_tot + K.epsilon())

In [ ]:
def floorer(t):
    return math.floor(t)

In [ ]:
def rmse(y_true, y_pred):
    from keras import backend

    return backend.sqrt(backend.mean(backend.square(y_pred - y_true), axis=-1))

In [ ]:
class kerasRegressor_regression:
    "kerasRegressor_regression"
    algorithm_implementation = KerasRegressor(
        build_fn=create_model, verbose=0, learning_rate=0.0001, layers=1, width=1
    )
    method_name = "kerasRegressor_regression"
    X_data = X
    y_data = y

    # vals = np.linspace(2, 750, 6)
    vals = np.logspace(1, 2.0, 3)

    floorer = lambda t: math.floor(t)
    floored_width = np.array([floorer(xi) for xi in vals])
    floored_width = np.insert(floored_width, 0, 1, axis=None)
    floored_width

    vals = np.logspace(1, 2.0, 3)

    floorer = lambda t: math.floor(t)
    floored_depth = np.array([floorer(xi) for xi in vals])
    floored_depth = np.insert(floored_depth, 0, 1, axis=None)
    floored_depth

    parameter_space = {
        "layers": floored_depth,
        #'layers':[1],
        #'width':[1],
        "epochs": [300],  # log_large_long,
        "batch_size": [int(len(X) / 2)],
        # kernel_reg=reg_reg_list_comb,
        "width": floored_width,
        "learning_rate": np.logspace(-4, -6, 3),
        # dropout_val = np.logspace(-1, -3, 2)
    }

In [ ]:
# break

In [ ]:
model_class_list = [
    multiple_regression,
    ridge_regression,
    lasso_regression,
    linearSVR_regression,
    baggingregressor_regression,
    kneighborsregressor_regression,
    randomforestregressor_regression,
    gradientboostingregressor_regression,
    decisiontreeregressor_regression,
    SVR_regression,
    NuSVR_regression,
    MLPRegressor_regression,
    SVR_regression,
    AdaBoostRegressor_regression,
    QuantileRegressor_regression,
    ExtraTreesRegressor_regression,
    kerasRegressor_regression,
    XGBRegressor_linear_regression,
    XGBRegressor_tree_regression,
    kerasRegressor_regression,
]

# check grid sizes and validity

In [ ]:
for elem in model_class_list:
    pg = ParameterGrid(elem.parameter_space)
    print(elem.method_name)
    print(len(pg))

In [ ]:
#break

In [ ]:
elem

In [ ]:
i = 1

In [ ]:
%%time
# class_name = model_class_list[i]

class_name = gradientboostingregressor_regression

scores = grid_search_crossvalidate(
    class_name.algorithm_implementation,
    class_name.parameter_space,
    class_name.method_name,
    class_name.X_data,
    class_name.y_data,
)
i = i + 1

In [ ]:
scores

In [ ]:
scores_tuple_list[0]

In [ ]:
scores_tuple_list[0][2].best_params_

In [ ]:
XGBRegressor_tree_regression.algorithm_implementation(parameters = scores_tuple_list[1][2].best_params_)

In [ ]:
best_model = xgb.XGBRegressor(**scores_tuple_list[1][2].best_params_)

best_model.fit(X, y)

In [ ]:
scores = cross_val_score(best_model, X, y, scoring='r2', cv=cv) 

In [ ]:
mean(scores)

In [ ]:
best_model = GradientBoostingRegressor(**scores_tuple_list[0][2].best_params_)

best_model.fit(sm.add_constant(X), y)

In [ ]:
scores = cross_val_score(best_model, X, y, scoring='r2', cv=cv) 

In [ ]:
scores

In [ ]:
X[0:1]

In [ ]:
best_model.predict(sm.add_constant(X)[0:1])

In [ ]:
with open("GradientBoostingRegressor_webm.pickle", "wb") as handle:
    pickle.dump(best_model, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
#break

In [ ]:
model_error_list = []

In [ ]:
scores_tuple_list = []

In [ ]:
multiprocess = False

In [ ]:
%%time
if multiprocess == False:
    for model_class in model_class_list:

        try:
            class_name = model_class

            grid_search_crossvalidate(
                class_name.algorithm_implementation,
                class_name.parameter_space,
                class_name.method_name,
                class_name.X_data,
                class_name.y_data,
            )
        except Exception as e:
            print("error on ", method_name)
            model_error_list.append((model_class.method_name, e))

In [ ]:
arg_list = []
for model_class in model_class_list:

    class_name = model_class

    arg_list.append(
        (
            class_name.algorithm_implementation,
            class_name.parameter_space,
            class_name.method_name,
            class_name.X_data,
            class_name.y_data,
        )
    )

In [ ]:
%%time
if multiprocess == True:

    def multi_run_wrapper(args):
        return grid_search_crossvalidate(*args)

    if __name__ == "__main__":
        from multiprocessing import Pool

        pool = Pool(4)
        results = pool.map(multi_run_wrapper, arg_list)
        # print(results)

In [ ]:
with open(plot_path + "regressor_grid_scores.pickle", "wb") as handle:
    pickle.dump(scores_tuple_list, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
with open(plot_path + "regressor_grid_scores.pickle", "rb") as handle:
    regressor_grid_scores = pickle.load(handle)

In [ ]:
model_error_list

In [ ]:
scores_tuple_list

In [ ]:
len(scores_tuple_list)

In [ ]:
len(model_class_list)

In [ ]:
for algo in scores_tuple_list:
    print(algo[0], mean(algo[1]["fit_time"]))

In [ ]:
for algo in scores_tuple_list:
    print(algo[0], mean(algo[1]["test_r2"]))

In [ ]:
for algo in scores_tuple_list:
    print(algo[0], max(algo[1]["test_r2"]))

In [ ]:
algo